## Summary of D22 - Label Encoding and One-hot Encoding

### [one hot vs label encoding](https://tw.saowen.com/a/1b61dc9734ec055fd42ce2160acc79171cc5ff121248c3ceb44d65c4235feb57)

 - One hot encoding: 
     
獨熱碼，在英文文獻中稱做 one-hot code, 直觀來說就是有多少個狀態就有多少位元，而且只有一個位元為1，其他全為0的一種碼制。

```
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])    # fit來學習編碼
enc.transform([[0, 1, 3]]).toarray()    # 進行編碼
# out: array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

# OR

pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)
```

__pros__:獨熱編碼解決了分類器不好處理屬性資料的問題，在一定程度上也起到了擴充特徵的作用。它的值只有0和1，不同的型別儲存在垂直的空間

__cons__:     當類別的數量很多時，特徵空間會變得非常大。在這種情況下，一般可以用PCA來減少維度。而且one hot encoding+PCA這種組合在實際中也非常有用。

__not use__: 有些基於樹的演算法(__Tree Model__)在處理變數時，並不是基於向量空間度量，數值只是個類別符號，即沒有偏序關係，所以不用進行獨熱編碼

- Label encoding:

```
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit([1,5,67,100])
le.transform([1,1,100,67,5])
```

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

In [6]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [10]:
## Estimator
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

estimator1 = LinearRegression()
estimator2 = GradientBoostingRegressor()

In [20]:
## Label Encoder
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]

score_lr = cross_val_score(estimator1, train_X, train_Y, cv=5).mean()
score_gbr = cross_val_score(estimator2, train_X, train_Y, cv=5).mean()

print('Scores of Label Encoder w/ linear regression: {}'.format(score_lr))
print('Scores of Label Encoder w/ gradient boosting regressor: {}'.format(score_gbr))

Scores of Label Encoder w/ linear regression: 0.33494273995838764
Scores of Label Encoder w/ gradient boosting regressor: 0.3774755107984499


In [21]:
## One hot Encoder
df_temp = pd.get_dummies(df)
train_X2 = df_temp[:train_num]

score_lr = cross_val_score(estimator1, train_X2, train_Y, cv=5).mean()
score_gbr = cross_val_score(estimator2, train_X2, train_Y, cv=5).mean()

print('Scores of One Hot Encoder w/ linear regression: {}'.format(score_lr))
print('Scores of One Hot Encoder w/ gradient boosting regressor: {}'.format(score_gbr))

Scores of One Hot Encoder w/ linear regression: 0.4130754436483536
Scores of One Hot Encoder w/ gradient boosting regressor: 0.3769922500553754


## Ans:
one hot encoder performs better than label encoder on linear regression. However, gradient boosting is a tree model, which is not suitable for one hot encoding, therefore, label encoder performs slightly better than one hot encoder.

# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)

In [24]:
# 標籤編碼 + 羅吉斯迴歸
"""
Your Code Here
"""
estimator = LogisticRegression()
score_label = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print('Scores of Label Encoder w/ logistic regression: {}'.format(score_label))

Scores of Label Encoder w/ logistic regression: 0.780004837244799


In [26]:
# 獨熱編碼 + 羅吉斯迴歸

"""
Your Code Here
"""
score_onehot = cross_val_score(estimator, train_X2, train_Y, cv=5).mean()
print('Scores of One-hot Encoder w/ logistic regression: {}'.format(score_onehot))

Scores of One-hot Encoder w/ logistic regression: 0.8013346043513216


## Ans:
One-hot encoder performs better than label encoder on logistic regression